In [5]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import scipy.constants as sc
from pathlib import Path
import mpmath as mp


temperature = 300  # K

Angle = np.deg2rad(np.array([-14.92, -10.83, 30.79, -30.79, 10.83, 14.92, -14.91, -13.29, -53.16, 53.16, 13.29, 14.91]))
rotation = np.array([0, 1, 0, 1, 0, 2, 0, 3, 0, 3, 0, 2])
labels = {
    1: {'label': 'T-DPP', 'color': 'b'},
    2: {'label': 'T-T', 'color': 'm'},
    3: {'label': 'T-E', 'color': 'c'},
    # 4: {"label": "FT-FT", "color": 'g'},
    # 5: {"label": "ADTDI-FT", "color": 'r'},
}

In [6]:
def read_data(file_name):
    data = np.loadtxt(file_name)
    data = np.reshape(data, (-1, 2))
    if data[:, 0].max() - data[:, 0].min() != 360:
        mirrored = np.column_stack((-data[:, 0] + 360, data[:, 1]))
        combined = np.vstack((data, mirrored))
        combined = np.unique(combined, axis=0)
        return combined[np.argsort(combined[:, 0])]
    else:
        return data[np.argsort(data[:, 0])]


# Fit function
def fit_function(data_label):
    data = read_data(Path(f"{data_label['label']}.txt"))
    fitf = interp1d(data[:, 0],
                    data[:, 1],
                    kind='cubic',
                    fill_value="extrapolate")
    return fitf


# Compute rotation integrals with mp for precision
def compute_rotation_integrals(fitf, kTval):

    def exp_energy(phi_deg):
        phi_deg = mp.mpf(phi_deg)
        return mp.exp(-fitf(float(phi_deg)) / kTval)

    # Compute partition function Z and integrals
    Z = mp.quad(exp_energy, [0, 360])
    m_i = mp.quad(
        lambda phi_deg: mp.cos(mp.mpf(phi_deg * mp.pi / 180)) * exp_energy(
            phi_deg), [0, 360]) / Z
    s_i = mp.quad(
        lambda phi_deg: mp.sin(mp.mpf(phi_deg * mp.pi / 180)) * exp_energy(
            phi_deg), [0, 360]) / Z

    return m_i, s_i


# Make Mmat with rotation matrices
def make_Mmat(all_data, Angle_rad, rotation_types, temperature):
    kTval = sc.R * temperature / 1000  # in kJ/mol
    M = len(rotation_types)
    A_list = []
    # Cache for rotation integrals (rot_id -> (m_i, s_i))
    integral_cache = {}
    for i in range(M):
        rot_id = int(rotation_types[i])
        theta = float(Angle_rad[i])
        if rot_id == 0:
            m_i, s_i = 1.0, 0.0
        elif rot_id == -1:
            m_i, s_i = -1.0, 0.0
        else:
            if rot_id not in integral_cache:
                fitf = all_data[rot_id]
                m_i, s_i = compute_rotation_integrals(fitf, kTval)
                integral_cache[rot_id] = (m_i, s_i)
            else:
                m_i, s_i = integral_cache[rot_id]

        # Rotation matrix for a given type
        S = np.array([[1, 0.0, 0.0], [0.0, m_i, -s_i], [0.0, s_i, m_i]])
        c = np.cos(theta)
        s = np.sin(theta)
        R_z = np.array([[c, -s, 0.0], [s, c, 0.0], [0.0, 0.0, 1]])
        A_list.append(S @ R_z)

    # Multiply all A_i for the repeat unit
    Mmat = np.eye(3)
    for A in A_list:
        Mmat = A @ Mmat
    return Mmat


# Compute persistence length using mp
def compute_persistence_in_repeats(Mmat):
    """Calculates the persistence length in repeat units using mpmath for higher precision"""

    # Convert Mmat to mp matrix for high precision calculations
    Mmat_mp = mp.matrix(Mmat.tolist())  # Convert to mp matrix
    # If eigs is a tuple, take the first element which are the eigenvalues

    # Compute eigenvalues with mp for precision
    eigs = mp.eig(Mmat_mp)
    if isinstance(eigs, tuple):
        eigs = eigs[0]
    # Find the maximum eigenvalue
    lambda_max = max(abs(e) for e in eigs)

    # Check if the maximum eigenvalue is >= 1.0
    if lambda_max >= 1.0:
        return np.inf, 1.0

    # Calculate persistence length in repeat units
    lp_in_repeats = -1.0 / mp.log(lambda_max)

    # Convert the result back to a float for easier display
    return float(lp_in_repeats), float(lambda_max)

In [7]:
all_data = {}
for key, label_info in labels.items():
    all_data[key] = fit_function(label_info)
matrix = make_Mmat(all_data, Angle, rotation, temperature)

In [8]:
lp_repeats, lam = compute_persistence_in_repeats(matrix)
# deflection = np.arccos(lam)

print(f"Max eigen value: lambda_max = {lam:.12f}")
print(f"Persistence length (in repeat units) = {lp_repeats:.6e}")
# print(f"Deflection angle (in degrees) = {np.rad2deg(deflection):.6f}")

Max eigen value: lambda_max = 0.895482371669
Persistence length (in repeat units) = 9.058566e+00
